In [108]:

# Install all these libraries using pip command
# Here I made your work easy (use 'pip3' if on pip3 version): 
# pip install PyPDF2 langchain python-dotenv pandas chromadb google-api-python-client tqdm

import PyPDF2
import langchain
import os
from dotenv import load_dotenv
from pprint import pprint
import pandas as pd
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
import google.generativeai as genai
from IPython.display import Markdown
from chromadb.api.types import Embeddings
import time
from tqdm import tqdm
from google.generativeai import GenerationConfig, GenerativeModel

In [110]:
api_key = " Enter your key here, which you could get on GoogleAPI key for gemini "  # Change it with your API key
genai.configure(api_key=api_key)

In [331]:
# A very basic function to extract text from pdf

def extract_text(pdf_path):
    extracted_text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_number in range(num_pages):
            page = pdf_reader.pages[page_number]
            extracted_text += page.extract_text()
    print(extracted_text)
    return extracted_text

pdf_path = "/Users/mihiresh/Mihiresh/Hackathon/gdsc2/api_testing/assets/sna_b.pdf"  # Don't forget to give your address here 
text = extract_text(pdf_path)

SOCIAL NETWORK ANALYSISModule 2 : Social Network Structure , Measures & Visualization Faculty Name :Mrs Snehal PatilLecture 1 Basics of Social network StructureØBasics of Social network StructureØNetwork VisualizationØNetwork FeaturesØScale IssuesØSocial Media Network AnalyticsØCommon  Network termsØCommon Social Media Network typesModule 2 Contents
Module 2–Social Network Structure , Measures &Visualization3ØText Books:ØSeven Layers of Social Media Analytics_ Mining Business Insights from Social Media Text, Actions, Networks, Hyperlinks, Apps, Search Engine, and Location Data, Gohar F. Khan,(ISBN-10: 1507823207). ØAnalyzing the Social Web 1st Edition by Jennifer Golbeck ØMining the Social Web_ Analyzing Data from Facebook, Twitter, LinkedIn, and Other Social Media Sites, Matthew A Russell, O’Reilly ØReference Books:Ø Social Media Analytics [2015], Techniques and Insights for Extracting Business Value Out of Social Media, Matthew Ganis, AvinashKohirkar, IBM Press 74 ØSocial Media Analy

In [332]:
# text = text.replace("Module 1– Introduction to Social Network"," ")
# text

In [333]:
# The most important step of chunking your textual data. Here you could use complete creativity while fine-tuning
# My suggestion try performing content-based chunking


from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,    #for next time let's try to keep the chunk size little bigger
    chunk_overlap = 100,
    length_function = len,
    add_start_index = True
)

In [334]:
texts = text_splitter.create_documents([text])
print(texts[14].page_content)

through B.Ø Thus, the fraction for all of these is zero. Now we can calculate the betweenness for B:


In [335]:
# Combinig chunks of your textual data to form a List. 

docs = []

for chunk in texts:
    docs.append(chunk.page_content)

print(docs[:2])

['SOCIAL NETWORK ANALYSISModule 2 : Social Network Structure , Measures & Visualization Faculty Name :Mrs Snehal PatilLecture 1 Basics of Social network StructureØBasics of Social network StructureØNetwork VisualizationØNetwork FeaturesØScale IssuesØSocial Media Network AnalyticsØCommon  Network termsØCommon Social Media Network typesModule 2 Contents', 'Module 2–Social Network Structure , Measures &Visualization3ØText Books:ØSeven Layers of Social Media Analytics_ Mining Business Insights from Social Media Text, Actions, Networks, Hyperlinks, Apps, Search Engine, and Location Data, Gohar F. Khan,(ISBN-10: 1507823207). ØAnalyzing the Social Web 1st Edition by Jennifer Golbeck ØMining the Social Web_ Analyzing Data from Facebook, Twitter, LinkedIn, and Other Social Media Sites, Matthew A Russell, O’Reilly ØReference Books:Ø Social Media Analytics [2015], Techniques and Insights for Extracting Business Value Out of Social Media, Matthew Ganis, AvinashKohirkar, IBM Press 74 ØSocial Media 

In [336]:
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        model = 'models/embedding-001'
        title = 'Social Network Analysis'
        return genai.embed_content(
            model=model,
            content=input,
            task_type="retrieval_document",
            title=title)["embedding"]
        

In [337]:
# Function to define and create you Vector Databse of chromadb in Database folders

def create_chroma_db(docs,name):
    chroma_client = chromadb.PersistentClient(path="/Users/mihiresh/Mihiresh/Hackathon/gdsc2/api_testing/database") #Don't forget to change path
    db = chroma_client.get_or_create_collection(
        name=name, embedding_function=GeminiEmbeddingFunction())
    
    initial_size = db.count()
    for i, d in tqdm(enumerate(docs), total=len(docs), desc="Creating Chroma DB"):
        db.add(
            documents=d,
            ids=str(i + initial_size)
        )
        time.sleep(0.5)
    return db


def get_chroma_db(name):
    chroma_client = chromadb.PersistentClient(path="/Users/mihiresh/Mihiresh/Hackathon/gdsc2/api_testing/database") # Here as well 
    return chroma_client.get_collection(name=name, function=EmbeddingFunction())

In [338]:
# Creating your Vector Database. [Don't unneccessarily run this code]

db = create_chroma_db(docs, "sme_db")
db.count()

Creating Chroma DB: 100%|██████████| 52/52 [00:54<00:00,  1.04s/it]


1828

In [339]:
pd.DataFrame(db.peek(1573))

,ids,embeddings,metadatas,documents,uris,data
0,0,"[0.015172463841736317, -0.010413325391709805, ...",None,Python Tutorial\nRelease 3.7.0\nGuido van Ross...,None,None
1,1,"[0.03553083539009094, 0.01233371626585722, -0....",None,4.3 The range() Function . . . . . . . . . . ....,None,None
2,10,"[0.019464796409010887, -0.03204401955008507, -...",None,"and tools, and additional documentation.\nThe ...",None,None
3,100,"[-0.0029656183905899525, 0.003489450318738818,...",None,Yuck!\nC>\nThese two variables are only deﬁned...,None,None
4,1000,"[-0.03242751583456993, -0.009994188323616982, ...",None,"compare unequal (except with themselves), and ...",None,None
...,...,...,...,...,...,...
1568,765,"[-0.025212131440639496, 0.012208681553602219, ...",None,"example, Pascal or C); parentheses ( ()) can b...",None,None
1569,766,"[-0.003493468277156353, -0.01856415718793869, ...",None,>>>5**2# 5 squared\n25\n>>>2**7# 2 to the powe...,None,None
1570,767,"[-0.03434646874666214, -0.016499843448400497, ...",None,>>>tax=12.5/100\n>>>price=100.50\n>>>price*tax...,None,None
1571,768,"[-0.015795739367604256, -0.03558232635259628, ...",None,quotes:\n>>>'spam eggs '# single quotes\n'spam...,None,None


In [340]:
len(pd.DataFrame(db.peek(10)).iloc[0]["embeddings"])

768

In [389]:
# Function to retrieve fact from the vector database

def get_relevant_passages(query, db, n_results):
    passages = db.query(query_texts=[query], n_results=n_results)['documents'][0]
    return passages

In [390]:
model = genai.GenerativeModel('gemini-pro')

In [391]:
def make_prompt(ques, knowledge):
    text = knowledge.replace("'","").replace('"','') #even i dont know why i did this


    # Choose from the given prompts which suits you the BEST
    # If you want you may create your own prompt by referring to the following options
    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


    # prompt = f"""question: {ques}.\n
    # information base or knowledge base: {text}\n
    # Answer the question in 1250-2500 words strictly based on the information base or knowledge base given, \n
    # If the question is asking for a code then also explain the algorithm, if the question is asking for career guidance then provide complete career guidance and links for various courses, if difference is asked differentiate with minimum 7 points in tabular format and one example
    # if the information is not sufficient then give output as "Info not good to answer"
    # """

    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    # prompt = f"""question: {ques}.\n
    # information base or knowledge base: {text}\n
    # Answer the question strictly based from {text} by filtering the required information from {text}\n
    # if the information is not sufficient then use keywords from {text} related to {ques} and answer the queestion and at the end of answer mention "[from other sources as well]" BUT try to give answers from only and only{text}\n
    # If the question is asking for a code then also explain the algorithm,\n 
    # if the question is asking for career guidance then provide complete career guidance and links for various courses, \n
    # if difference is asked differentiate with minimum 7 points in tabular format and one example
    # MOST IMPORTANTLY: answer the questions from {text} in such a way to write in exam\n
    # NOTE: try to answwer {ques} from {text} as much as possible
    # """

    # --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

    prompt = f"""question: {ques}.\n
    information base or knowledge base: {text}\n
    check whether {ques} is a question or an instruction, if its an instruction give an affirmative response \n
    ELSE
    Answer the question STRICTLY based from {text} by filtering the required information from {text}\n
    if the information is not sufficient then use keywords from {text} related to {ques} and answer the queestion and at the end of answer mention "[from other sources as well]" BUT try to give answers from only and only{text}\n
    If the question is asking for a code then also explain the algorithm,\n 
    if the question is asking for career guidance then provide complete career guidance and links for various courses, \n
    if difference is asked differentiate with minimum 7 points in tabular format and one example
    MOST IMPORTANTLY: answer the questions from {text} in such a way to write in exam\n
    NOTE: try to answwer {ques} from {text} as much as possible
    """

    gen_config = GenerationConfig(temperature=0.2)
    answer = model.generate_content(prompt)
    return answer.text

In [392]:
ques = " Define Social Network Analysis " # Enter any question you want
passages = get_relevant_passages(ques, db, n_results=25) #i have kept the n_results more because i wanted more info to be included in my answwer
txt = ""
for passage in passages:
    txt += passage

In [393]:
def list_to_string(passages):
    content = ""
    for passage in passages:
        content += passage + "\n"
    return content



In [394]:
cont = list_to_string(passages)
cont = cont.replace('Module 2–Social Network Structure',' ') #this is subjective for this data
cont = cont.replace('Module 1– Introduction to Social Network',' ') #this is subjective for this data
cont = cont.replace('\n',' ')  #this is subjective for this data
cont

"at a workgroup level and organization level, focusing on the interplay between the two structures. Three levels of social network analysis  76  13•A social network is defined as a network of interactions or relationships, where the nodes consist of actors, and the edges consist of the relationships or interactions between these actors.•The advantage of social network analysis is that, unlike many other methods, it focuses on interaction (rather than on individual behavior).•Network analysis allows us to examine how the configuration of networks influences how individuals and groups, organizations, or systems function. Module 1: Social network: An Overview-Introduction  6Module Contents  71. Social network: An Overview Introduction, History of social network, need to study social network, Applications of social network, Three levels of social network analysis, Graph visualization Tool.Ø SNA has its origins both in science and in the broader fields of network analaysis and graph theory.

In [395]:
answer = make_prompt(ques, cont)
Markdown(answer)

Social Network Analysis (SNA) is a method for studying the structure and dynamics of social networks. It involves the collection and analysis of data on the relationships between actors in a social network, such as individuals, organizations, or groups. SNA can be used to understand how social networks operate, how they change over time, and how they influence the behavior of actors within the network.

SNA can be used to study a variety of social phenomena, including:

* The spread of information and influence
* The formation of social groups
* The development of social norms
* The emergence of social movements
* The impact of social networks on individual behavior

SNA can be used in a variety of settings, including:

* Business
* Government
* Education
* Healthcare
* Non-profit organizations

SNA can be a valuable tool for understanding social behavior and for developing interventions to address social problems.

**Three Levels of Social Network Analysis**

There are three levels of social network analysis:

* **Micro-level analysis** focuses on the relationships between individual actors within a social network. This level of analysis can be used to understand how individual actors interact with each other and how these interactions influence their behavior.
* **Meso-level analysis** focuses on the relationships between groups of actors within a social network. This level of analysis can be used to understand how groups form, how they interact with each other, and how they influence the behavior of their members.
* **Macro-level analysis** focuses on the overall structure of a social network. This level of analysis can be used to understand how the network is organized, how it changes over time, and how it influences the behavior of actors within the network.

**Workgroup Level and Organization Level**

SNA can be used to study social networks at the workgroup level and the organization level.

* **Workgroup-level SNA** can be used to understand how workgroups form, how they interact with each other, and how they influence the behavior of their members. This information can be used to improve team performance and collaboration.
* **Organization-level SNA** can be used to understand how the organization is structured, how it changes over time, and how it influences the behavior of employees. This information can be used to improve organizational efficiency and effectiveness.

**Interplay Between Workgroup and Organization Levels**

The interplay between the workgroup level and the organization level is important for understanding how social networks operate within organizations. Workgroups are often embedded within larger organizations, and the relationships between workgroups can have a significant impact on the overall functioning of the organization. For example, if there is a lack of communication and collaboration between workgroups, this can lead to problems such as duplication of effort, conflict, and missed deadlines.

SNA can be used to identify and address problems in the interplay between the workgroup level and the organization level. By understanding the relationships between workgroups and the organization as a whole, organizations can take steps to improve communication, collaboration, and coordination. This can lead to improved organizational performance and effectiveness.

[from other sources as well]

#### /Users/mihiresh/Mihiresh/Hackathon/gdsc2/api_testing/rag.ipynb